# **Final Project**

##**Problem stament :**     

The widespread dissemination of fake news and propaganda presents serious societal risks, including the erosion of public trust, political polarization, manipulation of elections, and the spread of harmful misinformation during crises such as pandemics or conflicts. From an NLP perspective, detecting fake news is fraught with challenges. Linguistically, fake news often mimics the tone and structure of legitimate journalism, making it difficult to distinguish using surface-level features. The absence of reliable and up-to-date labeled datasets, especially across multiple languages and regions, hampers the effectiveness of supervised learning models. Additionally, the dynamic and adversarial nature of misinformation means that malicious actors constantly evolve their language and strategies to bypass detection systems. Cultural context, sarcasm, satire, and implicit bias further complicate automated analysis. Moreover, NLP models risk amplifying biases present in training data, leading to unfair classifications and potential censorship of legitimate content. These challenges underscore the need for cautious, context-aware approaches, as the failure to address them can inadvertently contribute to misinformation, rather than mitigate it.



Use datasets in link : https://drive.google.com/drive/folders/1mrX3vPKhEzxG96OCPpCeh9F8m_QKCM4z?usp=sharing
to complete requirement.

## **About dataset:**

* **True Articles**:

  * **File**: `MisinfoSuperset_TRUE.csv`
  * **Sources**:

    * Reputable media outlets like **Reuters**, **The New York Times**, **The Washington Post**, etc.

* **Fake/Misinformation/Propaganda Articles**:

  * **File**: `MisinfoSuperset_FAKE.csv`
  * **Sources**:

    * **American right-wing extremist websites** (e.g., Redflag Newsdesk, Breitbart, Truth Broadcast Network)
    * **Public dataset** from:

      * Ahmed, H., Traore, I., & Saad, S. (2017): "Detection of Online Fake News Using N-Gram Analysis and Machine Learning Techniques" *(Springer LNCS 10618)*



## **Requirement**

A team consisting of three members must complete a project that involves applying the methods learned from the beginning of the course up to the present. The team is expected to follow and document the entire machine learning workflow, which includes the following steps:

1. **Data Preprocessing**: Clean and prepare the dataset,etc.

2. **Exploratory Data Analysis (EDA)**: Explore and visualize the data.

3. **Model Building**: Select and build one or more machine learning models suitable for the problem at hand.

4. **Hyperparameter set up**: Set and adjust the model's hyperparameters using appropriate methods to improve performance.

5. **Model Training**: Train the model(s) on the training dataset.

6. **Performance Evaluation**: Evaluate the trained model(s) using appropriate metrics (e.g., accuracy, precision, recall, F1-score, confusion matrix, etc.) and validate their performance on unseen data.

7. **Conclusion**: Summarize the results, discuss the model's strengths and weaknesses, and suggest possible improvements or future work.





In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
df = pd.read_csv(r"C:\Users\Admin\OneDrive - VNU-HCMUS\cleaned")
print(df.head())
print(df.shape)

                                                text  label  text_length
0  link imo happening guys jumping conclusions co...      1         2717
1  house passed budget today slim margin democrat...      1         3203
2  youtube video posted january cloudgate studio ...      0         2549
3  marseille france sang danced chanted even reas...      0         8149
4  sunday nbc meet press discussing president don...      0         1235
(68604, 3)


In [61]:
df = df.dropna(subset=['text'])

In [62]:
true_df = df[df['label'] == 1]
fake_df = df[df['label'] == 0]

In [63]:
# Sắp xếp theo độ dài của văn bản giảm dần
true_df = true_df.sort_values(by='text_length', ascending=False)
fake_df = fake_df.sort_values(by='text_length', ascending=False)

# Chia thành train, valid, test set
true_train_df = true_df.iloc[:20000]
true_valid_df = true_df.iloc[20000:25000]
true_test_df = true_df.iloc[25000:]

fake_train_df = fake_df.iloc[:20000]
fake_valid_df = fake_df.iloc[20000:25000]
fake_test_df = fake_df.iloc[25000:]

# Tokenize data

In [ ]:
# Tokenize dữ liệu
from transformers import BertTokenizerFast
from transformers import XLNetTokenizerFast

class Tokenizer:
    def __init__(self, model_name, max_length=128):
        if 'bert' in model_name:
            self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        elif 'xlnet' in model_name:
            self.tokenizer = XLNetTokenizerFast.from_pretrained(model_name)
        self.max_length = max_length

    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )
    
# Convert pandas DataFrames to HuggingFace Dataset objects
from datasets import Dataset
true_train_dataset = Dataset.from_pandas(true_train_df)
true_valid_dataset = Dataset.from_pandas(true_valid_df)
true_test_dataset = Dataset.from_pandas(true_test_df)

fake_train_dataset = Dataset.from_pandas(fake_train_df)
fake_valid_dataset = Dataset.from_pandas(fake_valid_df)
fake_test_dataset = Dataset.from_pandas(fake_test_df)

tokenizer = Tokenizer('bert-base-uncased')

true_train_dataset = true_train_dataset.map(tokenizer.tokenize_function)
true_valid_dataset = true_valid_dataset.map(tokenizer.tokenize_function)
true_test_dataset = true_test_dataset.map(tokenizer.tokenize_function)

fake_train_dataset = fake_train_dataset.map(tokenizer.tokenize_function)
fake_valid_dataset = fake_valid_dataset.map(tokenizer.tokenize_function)
fake_test_dataset = fake_test_dataset.map(tokenizer.tokenize_function)

true_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
true_valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
true_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

fake_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
fake_valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
fake_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

# Create dataloader

In [26]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Build and Load model

In [32]:
# Build or Load Model
from torch.optim import AdamW
from transformers import BertForSequenceClassification, XLNetForSequenceClassification

class ModelBuilder:
    def __init__(self, num_labels=2):
        self.num_labels = num_labels
        self.model = None

    def build_model(self, model_name):
        if 'bert' in model_name:
            self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        elif 'xlnet' in model_name:
            self.model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        return self.model
    
    def optimizer(self, learning_rate=5e-5):
        return AdamW(self.model.parameters(), lr=learning_rate)

In [31]:
model_builder = ModelBuilder()
bert_model = model_builder.build_model('bert-base-uncased')
optimizer = model_builder.optimizer()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train model

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, XLNetForSequenceClassification

trainer = Trainer(
    model=bert_model,
    args=TrainingArguments(
        output_dir='./results',
        evaluation_strategy='epoch',
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
    ),
    train_dataset=true_train_dataset,
    eval_dataset=true_valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer.tokenizer,
    optimizers=(optimizer, None)  # Use the optimizer we created
)
trainer.train()

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=true_test_dataset)
print(f"Evaluation results: {eval_results}")